In [74]:
# Conexão com SQL Server
import pandas as pd
import pyodbc

server = 'WILLIAN\SQLEXPRESS'
database = 'SQL_DB_1'

conexao = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                         f'SERVER={server};'
                         f'DATABASE={database};'
                         'Trusted_Connection=yes;')
cursor = conexao.cursor()

Carga dos DFS

In [75]:
Produto=pd.read_excel(r"C:\Aplicativos\Meus_Arquivos_Willian\Ambiente_Desenvolvimento\Arquivos_in\Produto.xlsx")
Categoria=pd.read_excel(r"C:\Aplicativos\Meus_Arquivos_Willian\Ambiente_Desenvolvimento\Arquivos_in\Categoria.xlsx")
Itens=pd.read_excel(r"C:\Aplicativos\Meus_Arquivos_Willian\Ambiente_Desenvolvimento\Arquivos_in\items.xlsx")
Ordens=pd.read_excel(r"C:\Aplicativos\Meus_Arquivos_Willian\Ambiente_Desenvolvimento\Arquivos_in\Ordens.xlsx")
Clientes=pd.read_csv(r"C:\Aplicativos\Meus_Arquivos_Willian\Ambiente_Desenvolvimento\Arquivos_in\Clientes.csv",delimiter=',')


In [76]:
Produto.head()

,ID,Name,Price,Id_Category
0,0,Sapato vermelho,96,0
1,0,Sapato vermelho,96,0
2,1,Bolsa mais que Velha,8000,0
3,2,Calça,75,0
4,3,Camisa apertadinha,170,0


In [77]:
Categoria.head()

,id,name
0,0,Moda e Acessórios
1,1,Cosméticos e Perfumaria
2,2,Eletrodomésticos
3,3,Livros
4,4,Celulares


In [78]:
Itens.head()

,id,order_id,product_id,quantity,total_price
0,0,7019,28,1,186
1,1,8827,29,1,110
2,2,3863,38,1,385
3,3,5002,37,1,493
4,4,6337,28,1,186


In [79]:
Ordens.head()

,id,created_at,customer_id,status
0,0,2019-08-28 03:36:31,91,entregue
1,1,2019-08-16 20:21:30,7,entregue
2,2,2019-07-11 11:00:06,26,pagamento_pendente
3,3,2019-08-27 21:04:41,92,pagamento_pendente
4,4,2019-11-03 19:58:37,63,entrega_pendente


In [80]:
Clientes.head()

,id,created_at,first_name,last_name,email,cell_phone,country,state,street,number,additionals
0,0,2017-11-01 14:45:41,Marta,Jesus,NaN,9 9102-7834,Brasil,Acre,NaN,NaN,Conjunto 16
1,1,2017-10-16 00:50:39,Luana,Almeida,NaN,9 7328-8718,Brasil,Rio Grande do Sul,Avenida 56 do Estado Rio Grande do Sul,989.0,Conjunto 17
2,2,2018-06-16 17:51:29,Frida,Mendes,frida@meu_email.com,9 5906-7552,Brasil,São Paulo,Avenida 59 do Estado São Paulo,534.0,NaN
3,3,2018-01-17 03:02:58,Daniela,Avelino,daniela@exemplo.com,9 4642-9486,Brasil,Mato Grosso,NaN,NaN,NaN
4,4,2018-08-06 07:24:16,Romário,Teixeira,NaN,9 3093-6522,Brasil,Bahia,Praça 56 do Estado Bahia,191.0,Apto 12


Deleta dados das tabelas

In [81]:
cursor.execute(

"""
truncate table Produtos;
truncate table Categoria;
truncate table Items;
truncate table clientes;
truncate table Ordens;

""") # Apagar todos os dados
cursor.commit()


In [82]:
#Carga Tabela Produtos
for index, linha in Produto.iterrows():
    cursor.execute("Insert into [Produtos](ID, Nome, Price, Id_Category) values(?, ?, ?, ?)",linha.ID, linha.Name, linha.Price, linha.Id_Category)
cursor.commit() # validar dados no SQL Server

In [83]:
#Carga Tabela Categoria
for index, linha in Categoria.iterrows():
    cursor.execute(
        "INSERT INTO [Categoria](id, Categoria) VALUES (?, ?)",(linha['id'], linha['name'])
    )
cursor.commit()

In [84]:
#Carga Tabela Ordens
for index, linha in Ordens.iterrows():
    cursor.execute(
        "INSERT INTO [Ordens](id,created_at,customer_id,status) VALUES (?,?,?,?)",linha.id, linha.created_at,linha.customer_id,linha.status)
cursor.commit()

In [85]:
#Carga Tabela items
#Converte coluna total_price para float64
Itens['total_price'] = Itens['total_price'].astype('float64') # Converte para float64
for index, linha in Itens.iterrows():
    cursor.execute(
        "INSERT INTO [items](id,order_id,product_id,quantity,total_price) VALUES (?,?,?,?,?)",linha.id, linha.order_id,linha.product_id,linha.quantity,linha.total_price)
cursor.commit()

In [86]:
#Carga Tabela Clientes

#Converte coluna total_price para float64
Clientes['created_at'] = pd.to_datetime(Clientes['created_at'], errors='coerce') # - errors='coerce': diz ao pandas para forçar a conversão e, se encontrar algum valor inválido (como uma string malformada ou nula), ele vai substituir por NaT (Not a Time), que é o equivalente a NaN para datas.
# Se contrar valores NaT, você pode optar por removê-los ou preenchê-los com um valor padrão.
Clientes['email'] = Clientes['email'].fillna('Sem Registros')  # Se encontrar valores NaN, substitui por 'Sem Registros
Clientes['street'] = Clientes['street'].fillna('Sem Registros')
Clientes['number'] = Clientes['number'].fillna('Sem Registros')
Clientes['additionals'] = Clientes['additionals'].fillna('Sem Registros')

for index, linha in Clientes.iterrows():
    linha.email = str(linha.email) # Convertendo para string pois tem caracteres especiais
    linha.state = str(linha.state)
    linha.street = str(linha.street)
    linha.number = str(linha.number)
    linha.additionals = str(linha.additionals)
    cursor.execute(
        "INSERT INTO [Clientes](id,created_at,first_name,last_name,email,cell_phone,country,state,street,number,additionals) VALUES (?,?,?,?,?,?,?,?,?,?,?)",
        linha.id,
        linha.created_at,
        linha.first_name,
        linha.last_name,
        linha.email,
        linha.cell_phone,
        linha.country,
        linha.state,
        linha.street,
        linha.number,
        linha.additionals)
cursor.commit()

In [87]:
#Fechar todas as conexões
print("Todas as Tabela foram carregadas com êxito.")
cursor.close()
conexao.close()

Todas as Tabela foram carregadas com êxito.


In [88]:
print("Todas as Tabela foram carregadas com êxito.")

Todas as Tabela foram carregadas com êxito.
